<a href="https://colab.research.google.com/github/custom-hyper/CoinGeckoAPI/blob/main/CoinGecko_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#@title Install Libraries { display-mode: "code" }
!pip install plotly
!pip install pytrends
!pip install pycoingecko
!pip install pandas
!pip install tweepy
!pip install requests
!pip install psycopg2

## Setup SQLite


In [3]:
#@title Important Directories { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from pycoingecko import CoinGeckoAPI
import sqlite3 as sq
import hashlib

import logging
import sys
from tqdm.notebook import tqdm

In [4]:
#@title  Connect { display-mode: "code" }
path = '/content/drive/MyDrive/workspace/pipelines/db/' #@param {type: 'string'
database = "db.sqlite" #@param {type: 'string'
conn_SQLite = sq.connect(path+database)

In [5]:
#@title   Show Available Tables { display-mode: "code" }
cursor = conn_SQLite.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('deFi_lama_total_TVL',),
 ('proprietary_Twitter_id',),
 ('proprietary_tweets_extract',),
 ('proprietary_Twitter_partnerships_announcements',),
 ('proprietary_airtable_watch_all',),
 ('proprietary_Twitter_Partnership_ohlc',),
 ('proprietary_master_id',),
 ('CoinGecko_ohlc',),
 ('deFi_lama_general',),
 ('deFi_lama_tvl',)]

# CoinGecko API Statistics Report

In [6]:
#General Information table

def currency_stats(url):
    """
    Generates a list of the currencies on CoinGecko
    Args: url of the CoinGecko website

    """
    frames = []
    response = requests.get(url)
    data = response.json()

    df_currency = pd.DataFrame(data, columns = ['id'])
    """
    print('Stats: ')
    print(df_currency.info(verbose=True))
    print(df_currency.isnull().sum())
    print(data)

    print(df_currency.describe())
    """
    return df_currency

def main():

    url = "https://api.coingecko.com/api/v3/simple/supported_vs_currencies"
    currency_stats(url)
    
if __name__ == "__main__":
    main()



In [7]:

url = "https://api.coingecko.com/api/v3/coins/list"


frames = []
response = requests.get(url)
data = response.json()


token_list = pd.DataFrame(data)
token_list_clean = pd.DataFrame() 
token_list_clean['CoinGecko_id'] = token_list['id']
symbol = token_list['symbol'].str.upper()
name = token_list['name'] 

token_list_clean['prop_id'] = symbol + '.' + name.str.replace(" ", "")
token_list_clean['symbol'] = symbol
token_list_clean['name'] = name

token_list_clean['prop_timestamp'] = int( time.time_ns() / 1000 )

list_anonymous = token_list_clean['prop_id']

for i in list_anonymous:
    text = i
    hash_object = hashlib.md5(text.encode())
    md5_hash = hash_object.hexdigest()
    token_list_clean['anon_id'] = str('ANON') + str(md5_hash.upper())



token_list_clean = token_list_clean.dropna()
token_list_clean.reset_index(drop=True, inplace=True)
token_list_clean.head()


    

,CoinGecko_id,prop_id,symbol,name,prop_timestamp,anon_id
0,01coin,ZOC.01coin,ZOC,01coin,1650757778765588,ANON8E73CBF79B0030CB158A7E48275337D8
1,0-5x-long-algorand-token,ALGOHALF.0.5XLongAlgorandToken,ALGOHALF,0.5X Long Algorand Token,1650757778765588,ANON8E73CBF79B0030CB158A7E48275337D8
2,0-5x-long-altcoin-index-token,ALTHALF.0.5XLongAltcoinIndexToken,ALTHALF,0.5X Long Altcoin Index Token,1650757778765588,ANON8E73CBF79B0030CB158A7E48275337D8
3,0-5x-long-ascendex-token-token,ASDHALF.0.5XLongAscendExTokenToken,ASDHALF,0.5X Long AscendEx Token Token,1650757778765588,ANON8E73CBF79B0030CB158A7E48275337D8
4,0-5x-long-bitcoin-cash-token,BCHHALF.0.5XLongBitcoinCashToken,BCHHALF,0.5X Long Bitcoin Cash Token,1650757778765588,ANON8E73CBF79B0030CB158A7E48275337D8


In [8]:
#@title Load Token ID to SQLite { display-mode: "code" }

proprietary_master_id = "proprietary_master_id"

token_list_clean.to_sql(proprietary_master_id, conn_SQLite, if_exists='append', index=False) # writes to file
conn_SQLite.commit()

# Just be sure any changes have been committed or they will be lost.

df_master_id = pd.read_sql('select distinct * from {}'.format(proprietary_master_id), conn_SQLite)

In [9]:
df_master_id


,CoinGecko_id,prop_id,symbol,name,prop_timestamp,anon_id
0,01coin,ZOC.01coin,ZOC,01coin,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
1,0-5x-long-algorand-token,ALGOHALF.0.5XLongAlgorandToken,ALGOHALF,0.5X Long Algorand Token,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
2,0-5x-long-altcoin-index-token,ALTHALF.0.5XLongAltcoinIndexToken,ALTHALF,0.5X Long Altcoin Index Token,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
3,0-5x-long-ascendex-token-token,ASDHALF.0.5XLongAscendExTokenToken,ASDHALF,0.5X Long AscendEx Token Token,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
4,0-5x-long-bitcoin-cash-token,BCHHALF.0.5XLongBitcoinCashToken,BCHHALF,0.5X Long Bitcoin Cash Token,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
...,...,...,...,...,...,...
68434,zyro,ZYRO.Zyro,ZYRO,Zyro,1650757778765588,ANON8E73CBF79B0030CB158A7E48275337D8
68435,zyrri,ZYR.Zyrri,ZYR,Zyrri,1650757778765588,ANON8E73CBF79B0030CB158A7E48275337D8
68436,zytara-dollar,ZUSD.ZytaraDollar,ZUSD,Zytara Dollar,1650757778765588,ANON8E73CBF79B0030CB158A7E48275337D8
68437,zyx,ZYX.ZYX,ZYX,ZYX,1650757778765588,ANON8E73CBF79B0030CB158A7E48275337D8


# CoinGecko API Extraction

In [10]:
#@title  Download Coins Data Reports { display-mode: "code" }


url = "https://api.coingecko.com/api/v3/coins/list"
frames = []
response = requests.get(url)
data = response.json()
token_list = pd.DataFrame(data)
id = token_list.id
id


#loop data extraction to token list
try:

  start = 12001
  ends =   13500

  for i in tqdm(id[start : ends]):

    response = requests.get("https://api.coingecko.com/api/v3/coins/{}".format(i))
    output = response.text
    json_parsed = json.loads(output)
    

    #Convert data to list of dataframes
    full_table = pd.json_normalize(json_parsed)       
    #results = [full_table]
    frames.append(full_table)

    #Slow down requests

    time.sleep(60/50)
    #Status report
    logging.info('Page '+ str(i) + ' – Rows ' + ' – source: '+ str(url))
except ConnectionError:
    time.sleep(60)
 
single = pd.concat(frames) #Careful, this line must be outside of the for loop
single['prop_timestamp'] = int( time.time_ns() / 1000 )

print(single)
path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output/"
single.to_csv(path + 'CoinGecko_general_data_coins_{}-{}.csv'.format(start,ends))



  0%|          | 0/1499 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


                       id  symbol                   name    asset_platform_id  \
0            timelockcoin     tym           TimeLockCoin                 None   
0           time-new-bank     tnb          Time New Bank             ethereum   
0                  timerr  timerr                 Timerr  binance-smart-chain   
0                  timers     ipm                 Timers             ethereum   
0              timexspace     txs             TIMEXSPACE  binance-smart-chain   
..                    ...     ...                    ...                  ...   
0                    your    your                   Your                 None   
0     your-open-metaverse     yom    Your Open Metaverse                 None   
0                 youswap     you                YouSwap          huobi-token   
0             youves-uusd    uusd            Youves uUSD                tezos   
0   youves-you-governance     you  Youves YOU Governance                tezos   

    block_time_in_minutes h

In [11]:
single.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1499 entries, 0 to 0
Columns: 1780 entries, id to prop_timestamp
dtypes: float64(1300), int64(13), object(467)
memory usage: 20.4+ MB


# Aggregating Output

In [16]:
import os
import glob
import pandas as pd
import sys
from pathlib import Path




path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/bulk/"


Cleaned_data_file = 'Bulk_CoinGecko_coins.csv'

df = pd.DataFrame()

extension = 'csv'
#dataset_location = os.chdir(r'D:\investing_programs\datasets\CoinMetrics')
all_filenames = []
all_filenames = [i for i in glob.glob('/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output/*.{}'.format(extension))]

#combine all files in the list
for i in all_filenames:
    df = pd.read_csv(i)#.read(),engine='openpyxl')   
    print(df)

combined_csv = df.append([pd.read_csv(f) for f in all_filenames])


combined_csv.to_csv( path + Cleaned_data_file, index=False, encoding='utf-8-sig')


print(combined_csv.head())



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (170,1648,1680,1688,1707,1717,1724,1725,1730,1739,1743,1744,1746,1747,1749,1751,1752,1753,1754,1756,1757) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


     Unnamed: 0                                id    symbol  \
0             0                            01coin       zoc   
1             0          0-5x-long-algorand-token  algohalf   
2             0     0-5x-long-altcoin-index-token   althalf   
3             0    0-5x-long-ascendex-token-token   asdhalf   
4             0      0-5x-long-bitcoin-cash-token   bchhalf   
..          ...                               ...       ...   
995           0  aston-martin-cognizant-fan-token        am   
996           0             aston-villa-fan-token       avl   
997           0              astra-guild-ventures       agv   
998           0                       astral-farm    astral   
999           0                          astralis      STAR   

                                 name    asset_platform_id  \
0                              01coin                  NaN   
1            0.5X Long Algorand Token             ethereum   
2       0.5X Long Altcoin Index Token             ethereu

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (109,1690,1694,1715,1727,1729,1736,1740,1744,1747,1748,1753,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


     Unnamed: 0                id   symbol            name  \
0             0  astra-protocol-2    $astr  Astra Protocol   
1             0        astrobirdz      abz      AstroBirdz   
2             0       astrodonkey     dnky     AstroDonkey   
3             0         astroelon  elonone       AstroElon   
4             0        astrofarms      leo      AstroFarms   
..          ...               ...      ...             ...   
994           0    bone-shibaswap     bone  Bone ShibaSwap   
995           0          boneswap     bone        BoneSwap   
996           0        bone-token     bone      Bone Token   
997           0         bonezyard      bnz       BonezYard   
998           0             bonfi      bnf           BonFi   

       asset_platform_id  block_time_in_minutes hashing_algorithm  \
0                    NaN                      0               NaN   
1    binance-smart-chain                      0               NaN   
2    binance-smart-chain                      0 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1429,1703,1725,1727,1730,1733,1734,1735,1737,1738,1740,1741,1744,1745,1746,1749,1750,1751,1752,1753,1754,1756,1757,1758,1759,1760,1761,1762,1763) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

     Unnamed: 0                    id   symbol                  name  \
0             0              solapoly     slpy              Solapoly   
1             0                 solar    solar                 Solar   
2             0             solarbeam    solar             Solarbeam   
3             0            solar-bear  solbear            Solar Bear   
4             0          solar-energy      seg          Solar Energy   
..          ...                   ...      ...                   ...   
994           0           tiger-token     tgnb           Tiger Token   
995           0            tiki-token     tiki            Tiki Token   
996           0               tilwiki      tlw               TilWiki   
997           0  timechain-swap-token      tcs  Timechain Swap Token   
998           0     timecoin-protocol     tmcn     Timecoin Protocol   

       asset_platform_id  block_time_in_minutes hashing_algorithm  \
0                 solana                      0               NaN 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1716,1727,1730,1732,1734,1737,1739,1742,1745,1746,1747,1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,1760,1761,1763,1764,1765,1766,1768,1769,1770,1771,1772,1773,1774,1776,1777,1778) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


      Unnamed: 0                     id  symbol                   name  \
0              0           timelockcoin     tym           TimeLockCoin   
1              0          time-new-bank     tnb          Time New Bank   
2              0                 timerr  timerr                 Timerr   
3              0                 timers     ipm                 Timers   
4              0             timexspace     txs             TIMEXSPACE   
...          ...                    ...     ...                    ...   
1494           0                   your    your                   Your   
1495           0    your-open-metaverse     yom    Your Open Metaverse   
1496           0                youswap     you                YouSwap   
1497           0            youves-uusd    uusd            Youves uUSD   
1498           0  youves-you-governance     you  Youves YOU Governance   

        asset_platform_id  block_time_in_minutes hashing_algorithm  \
0                     NaN                

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DtypeWarning: Columns (170,1648,1680,1688,1707,1717,1724,1725,1730,1739,1743,1744,1746,1747,1749,1751,1752,1753,1754,1756,1757) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DtypeWarning: Columns (109,1690,1694,1715,1727,1729,1736,1740,1744,1747,1748,1753,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DtypeWarning: Columns (1429,1703,1725,1727,1730,1733,1734,1735,1737,1738,1740,1741,1744,1745,1746,1749,1750,1751,1752,1753,1754,1756,1757,1758,1759,1760,1761,1762,1763) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DtypeWarning: Columns (171,1454,1481,1553,1701,1709,1710,1714,1718,1725,1726,1731,1736,1737

   Unnamed: 0             id  symbol           name    asset_platform_id  \
0           0   timelockcoin     tym   TimeLockCoin                  NaN   
1           0  time-new-bank     tnb  Time New Bank             ethereum   
2           0         timerr  timerr         Timerr  binance-smart-chain   
3           0         timers     ipm         Timers             ethereum   
4           0     timexspace     txs     TIMEXSPACE  binance-smart-chain   

   block_time_in_minutes hashing_algorithm                         categories  \
0                      1             Quark                                 []   
1                      0               NaN                                 []   
2                      0               NaN  ['Binance Smart Chain Ecosystem']   
3                      0               NaN                                 []   
4                      0               NaN                                 []   

  public_notice                                 addition

In [13]:
combined_csv.describe()

,Unnamed: 0,block_time_in_minutes,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,...,platforms.nem,platforms.bitshares,platforms.stratis,ico_data.links.,platforms.omni,platforms.metaverse-etp,platforms.nxt,platforms.enq-enecuum,platforms.factom,platforms.openledger
count,14987.0,1.498700e+04,6099.000000,6099.000000,4189.000000,14984.000000,14987.000000,14987.000000,14987.000000,14987.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,0.0,5.499266e+02,71.855621,28.144389,1909.132490,6804.884410,7.760879,5.026033,10.651956,5.839835,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.0,4.181595e+04,35.036532,35.036530,1135.935787,3944.900112,9.116098,13.919848,9.726789,11.648047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.0,-2.000000e+00,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.0,0.000000e+00,50.000000,0.000000,953.000000,3381.750000,1.744000,0.000000,6.089000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.0,0.000000e+00,87.500000,12.500000,1899.000000,6774.000000,3.870000,0.000000,8.185000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,0.0,0.000000e+00,100.000000,50.000000,2862.000000,10227.250000,11.826000,0.000000,10.242000,2.805000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,0.0,3.600000e+06,100.000000,100.000000,13254.000000,13681.000000,80.443000,98.949000,83.577000,105.681000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
combined_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14987 entries, 0 to 1498
Columns: 1809 entries, Unnamed: 0 to platforms.openledger
dtypes: float64(1455), int64(12), object(342)
memory usage: 207.0+ MB


In [15]:
combined_csv.tail()

,Unnamed: 0,id,symbol,name,asset_platform_id,block_time_in_minutes,hashing_algorithm,categories,public_notice,additional_notices,...,platforms.stratis,ico_data.links.,ico_data.links.facebeook,platforms.omni,platforms.metaverse-etp,platforms.nxt,platforms.enq-enecuum,platforms.factom,platforms.rootstock,platforms.openledger
1494,0,your,your,Your,NaN,0,NaN,[],NaN,['No active trades are found for this coin. Pl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1495,0,your-open-metaverse,yom,Your Open Metaverse,NaN,0,NaN,[],NaN,['No active trades are found for this coin. Pl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1496,0,youswap,you,YouSwap,huobi-token,0,NaN,[],NaN,"[""Kindly be aware of <a href='https://www.coin...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1497,0,youves-uusd,uusd,Youves uUSD,tezos,0,NaN,[],NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1498,0,youves-you-governance,you,Youves YOU Governance,tezos,0,NaN,[],NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
